# Question Answering on SQL Database using OpenAI's Large Language Model (LLM) and langchain

In [4]:
# %pip install openai
# %pip install langchain
# %pip install sqlalchemy

In [5]:
import os
from sqlalchemy import create_engine
from langchain import OpenAI
from langchain.chains import ConversationChain

#### Connect to the Database using SQLAlchemy

In [6]:
conn_string = (
    "sqlite:///../database/moviesdb.db"
    )

sqlite_engine = create_engine(conn_string, echo=False) #echo can be set to true if you want to see intermediate processing messages as well

print('connection is ok')
print(sqlite_engine.table_names())

connection is ok
['movie_characters', 'movie_conversations', 'movie_lines', 'movie_script_urls', 'movie_titles']


C:\Users\hpune\AppData\Local\Temp\ipykernel_37420\1087331391.py:8: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  print(sqlite_engine.table_names())


#### Make Sure the below environment variables are set before connecting to OpenAI, otherwise the command will fail
OPENAI_API_KEY - OpenAI Api key. Create if from https://beta.openai.com

If not present in environment variables, it can also be set in notebook using below line.

os.environ["OPENAI_API_KEY"] = key_value

In [7]:
# first initialize the large language model
llm = OpenAI(
    temperature=0,
    model_name="text-davinci-003"
)


In [8]:
# Since we are going to ask questions based on sql database and want answers related to that, hence we are using SQLDatabaseChain from langchain
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

In [9]:
# We can set the list of tables which can be included for querying and whose schema can be passed to LLM (davinci model in our case)
include_tables = ['movie_characters', 'movie_conversations', 'movie_lines', 'movie_script_urls', 'movie_titles']

db = SQLDatabase(engine=sqlite_engine,
                 sample_rows_in_table_info=2, # We can set the number of samples to be included from database table to help LLM understand the table data better
                 include_tables=include_tables,  
                )

#### The table info shown below will be passed later to LLM so that it can answer our queries


In [18]:
print(db.get_table_info())


CREATE TABLE movie_titles (
	movie_id TEXT, 
	movie_title TEXT, 
	year_of_release TEXT, 
	"IMDB_rating" FLOAT, 
	"IMDB_number_of_votes" BIGINT, 
	movie_genre TEXT, 
	PRIMARY KEY (movie_id)
)

/*
2 rows from movie_titles table:
movie_id	movie_title	year_of_release	IMDB_rating	IMDB_number_of_votes	movie_genre
m0	10 things i hate about you	1999	6.9	62847	['comedy' 'romance']
m1	1492: conquest of paradise	1992	6.2	10421	['adventure' 'biography' 'drama' 'history']
*/


CREATE TABLE movie_characters (
	character_id TEXT, 
	character_name TEXT, 
	movie_id TEXT, 
	movie_title TEXT, 
	gender TEXT, 
	position_in_credits TEXT, 
	PRIMARY KEY (character_id), 
	FOREIGN KEY(movie_id) REFERENCES movie_titles (movie_id)
)

/*
2 rows from movie_characters table:
character_id	character_name	movie_id	movie_title	gender	position_in_credits
u0	BIANCA	m0	10 things i hate about you	f	4
u1	BRUCE	m0	10 things i hate about you	?	?
*/


CREATE TABLE movie_script_urls (
	movie_id TEXT, 
	movie_title TEXT, 
	movie

In [19]:
# We can declare a SQLDatabaseChain object which can help us in answering queries in natural language
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True) # if verbose is set to False, it wont show intermediate actions it took.


In [20]:
# Asking a query to the LLM based on our database
result = db_chain.run("How many unique movies are there?")
result



> Entering new SQLDatabaseChain chain...
How many unique movies are there?
SQLQuery: SELECT COUNT(DISTINCT movie_title) FROM movie_titles;
SQLResult: [(617,)]
Answer: There are 617 unique movies.
> Finished chain.


' There are 617 unique movies.'

#### As can be seen from the response that LLM was able to generate the correct query and was able to answer correctly

In [118]:
try:
    result = db_chain.run("which movie has most conversations ? ")
    result
except Exception as e:
    print()
    print("ERROR OCCURED: -")
    print(e)



> Entering new SQLDatabaseChain chain...
which movie has most conversations ? 
SQLQuery: "SELECT movie_title, COUNT(*) AS conversations FROM movie_conversations 
INNER JOIN movie_titles ON movie_conversations.movie_id = movie_titles.movie_id 
GROUP BY movie_title ORDER BY conversations DESC LIMIT 5"
ERROR OCCURED: -
(sqlite3.OperationalError) near ""SELECT movie_title, COUNT(*) AS conversations FROM movie_conversations 
INNER JOIN movie_titles ON movie_conversations.movie_id = movie_titles.movie_id 
GROUP BY movie_title ORDER BY conversations DESC LIMIT 5"": syntax error
[SQL:  "SELECT movie_title, COUNT(*) AS conversations FROM movie_conversations 
INNER JOIN movie_titles ON movie_conversations.movie_id = movie_titles.movie_id 
GROUP BY movie_title ORDER BY conversations DESC LIMIT 5"]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


An exception occured in the previous query. For some reason, we see that the LLM generated the intermediate sql query with quotes (*"SELECT movie_title, COUNT(\*) AS conversations FROM movie_conversations 
INNER JOIN movie_titles ON movie_conversations.movie_id = movie_titles.movie_id 
GROUP BY movie_title ORDER BY conversations DESC LIMIT 5"*) , and hence it gave a syntax error and the process failed, it should have avoided the surrounded quotes.

In the next prompt, we can add specific instructions to not use quotes and we can see that it generated the result correctly once we did that. (We need to engineer the prompt which makes the LLM less error prone).

We can also change the default prompt used by langchain but we'll see that later in the notebook


In [31]:
result = db_chain.run("which movie has most conversations ? generate sql query without quotes")
result



> Entering new SQLDatabaseChain chain...
which movie has most conversations ? generate sql query without quotes
SQLQuery: SELECT movie_title, COUNT(*) AS conversations_count FROM movie_conversations JOIN movie_titles ON movie_conversations.movie_id = movie_titles.movie_id GROUP BY movie_title ORDER BY conversations_count DESC LIMIT 5;
SQLResult: [('casino', 1530), ('chill factor', 1398), ('clerks.', 1286), ('jackie brown', 1214), ('all about eve', 1187)]
Answer: The movie with the most conversations is Casino with 1530 conversations.
> Finished chain.


' The movie with the most conversations is Casino with 1530 conversations.'

#### To Obtain intermediate step details as well, we can use below code instead. (In case if we want to access intermediate sql query or results via code)


In [32]:
db_chain_interim = SQLDatabaseChain(llm=llm, database=db, verbose=True, return_intermediate_steps=True)
result_full = db_chain_interim("which movie has most conversations ? generate sql query without quotes")
result_full



> Entering new SQLDatabaseChain chain...
which movie has most conversations ? generate sql query without quotes
SQLQuery:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


 SELECT movie_title, COUNT(*) AS conversations_count FROM movie_conversations JOIN movie_titles ON movie_conversations.movie_id = movie_titles.movie_id GROUP BY movie_title ORDER BY conversations_count DESC LIMIT 5;
SQLResult: [('casino', 1530), ('chill factor', 1398), ('clerks.', 1286), ('jackie brown', 1214), ('all about eve', 1187)]
Answer: The movie with the most conversations is Casino with 1530 conversations.
> Finished chain.


{'query': 'which movie has most conversations ? generate sql query without quotes',
 'result': ' The movie with the most conversations is Casino with 1530 conversations.',
 'intermediate_steps': [' SELECT movie_title, COUNT(*) AS conversations_count FROM movie_conversations JOIN movie_titles ON movie_conversations.movie_id = movie_titles.movie_id GROUP BY movie_title ORDER BY conversations_count DESC LIMIT 5;',
  "[('casino', 1530), ('chill factor', 1398), ('clerks.', 1286), ('jackie brown', 1214), ('all about eve', 1187)]"]}

In [35]:
# Obtaining intermediate query and intermediate results
intermediate_query= result_full.get("intermediate_steps", [""])[0]
intermediate_result= result_full.get("intermediate_steps", [""])[1]
print("Intermediate Query - ", intermediate_query)
print("Intermediate Result - ", intermediate_result)

Intermediate Query -   SELECT movie_title, COUNT(*) AS conversations_count FROM movie_conversations JOIN movie_titles ON movie_conversations.movie_id = movie_titles.movie_id GROUP BY movie_title ORDER BY conversations_count DESC LIMIT 5;
Intermediate Result -  [('casino', 1530), ('chill factor', 1398), ('clerks.', 1286), ('jackie brown', 1214), ('all about eve', 1187)]


In [39]:
#  Converting intermediate results into pandas dataframe
import pandas as pd
import re
columns = intermediate_query.split('FROM')[0].split('SELECT')[1].strip().split(",")

data_list = []
for substr in intermediate_result.split('),'):
    temp_str = re.sub("|\[|\]|'|\(|\)|\s", '', substr)
    temp_str = list(temp_str.split(','))
    data_list.append(temp_str)
df = pd.DataFrame(data_list, columns=columns)   
df

,movie_title,COUNT(*) AS conversations_count
0,casino,1530
1,chillfactor,1398
2,clerks.,1286
3,jackiebrown,1214
4,allabouteve,1187


In [49]:
result = db_chain.run("which movie are similiar to star wars based on rating and genre?, generate sql query without quotes")
result



> Entering new SQLDatabaseChain chain...
which movie are similiar to star wars based on rating and genre?, generate sql query without quotes
SQLQuery: SELECT movie_title, IMDB_rating, movie_genre FROM movie_titles WHERE movie_genre LIKE '%sci-fi%' AND IMDB_rating > 8.0 ORDER BY IMDB_rating DESC LIMIT 5
SQLResult: [('neuromancer', 9.3, "['action' 'adventure' 'animation' 'sci-fi']"), ('star wars', 8.8, "['action' 'adventure' 'fantasy' 'sci-fi']"), ('the matrix', 8.7, "['action' 'adventure' 'adventure' 'adventure' 'sci-fi']"), ('aliens', 8.5, "['action' 'sci-fi' 'thriller']"), ('eternal sunshine of the spotless mind', 8.5, "['drama' 'romance' 'sci-fi']")]
Answer: The top 5 movies similar to Star Wars based on rating and genre are Neuromancer, Star Wars, The Matrix, Aliens, and Eternal Sunshine of the Spotless Mind.
> Finished chain.


' The top 5 movies similar to Star Wars based on rating and genre are Neuromancer, Star Wars, The Matrix, Aliens, and Eternal Sunshine of the Spotless Mind.'

Above query was also returning error earlier so I had to add additional instruction in prompt to generate sql query without quotes.. Now lets see how we can modify the initial prompt itself and whether it is useful to do it or not in this case.

In [112]:
from langchain.prompts.prompt import PromptTemplate

_NEW_PROMPT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. 
Unless the user specifies in his question a specific number of examples he wishes to obtain, always limit your query to at most {top_k} results. 
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only look for the few relevant columns given the question.
Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Make sure only to answer questions based on given database tables and script urls unless specified later explicitly, also provide your sources in result. Generate sql query without quotes.
        Use the following format:

        Question: "Question here"
        SQLQuery: "SQL Query to run"
        SQLResult: "Result of the SQLQuery"
        Answer: "Final answer here along with source"

        Only use the tables listed below.
        {table_info}

        Question: {input}
        """

I modified the initial prompt a lot, added instruction to only answer question based on given database tables and also asked to give the sources, also added instruction to generate sql query without quotes. Lets see if it works

In [113]:
PROMPT = PromptTemplate(
            input_variables=["input", "table_info", "dialect", "top_k"], 
            template=_NEW_PROMPT_TEMPLATE
        )


In [114]:
new_db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True, prompt=PROMPT, top_k=5)
new_result = new_db_chain.run("which movie are similiar to star wars based on rating and genre?")



> Entering new SQLDatabaseChain chain...
which movie are similiar to star wars based on rating and genre?
SQLQuery: SELECT movie_title, IMDB_rating, movie_genre 
         FROM movie_titles 
         WHERE movie_genre LIKE '%sci-fi%' 
         AND IMDB_rating > 8 
         ORDER BY IMDB_rating DESC 
         LIMIT 5;

SQLResult: [('neuromancer', 9.3, "['action' 'adventure' 'animation' 'sci-fi']"), ('star wars', 8.8, "['action' 'adventure' 'fantasy' 'sci-fi']"), ('the matrix', 8.7, "['action' 'adventure' 'adventure' 'adventure' 'sci-fi']"), ('aliens', 8.5, "['action' 'sci-fi' 'thriller']"), ('eternal sunshine of the spotless mind', 8.5, "['drama' 'romance' 'sci-fi']")]
Answer: Movies similar to Star Wars based on rating and genre are Neuromancer (9.3), Star Wars (8.8), The Matrix (8.7), Aliens (8.5), and Eternal Sunshine of the Spotless Mind (8.5). Source: SQL query run on the tables provided.
> Finished chain.


Now I see that the same query is working and it listed the source also as SQL query which is much better than in other chain (db_chain) where it was failing as it added additional quotes around the SQL query.

However, I ran this same query multiple times and it sometimes gave me different results ocassionally (even when I have set temperature to 0). Ocassionally you'd notice that LLMs behave a bit randomly (thats something I have noticed). When we set temperature to 0 (it can be set between 0 to 1) , the randomness is reduced but still sometimes I noticed that. So, its possible you may see different results for same queries which I have run.

In [119]:
new_result = new_db_chain.run("what can you tell me about the movie 10 things i hate about you?")



> Entering new SQLDatabaseChain chain...
what can you tell me about the movie 10 things i hate about you?
SQLQuery:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).



SELECT movie_title, year_of_release, IMDB_rating, IMDB_number_of_votes, movie_genre 
FROM movie_titles 
WHERE movie_title = '10 things i hate about you' 
LIMIT 5;

SQLResult: [('10 things i hate about you', '1999', 6.9, 62847, "['comedy' 'romance']")]
Answer:
The movie 10 things i hate about you was released in 1999, has an IMDB rating of 6.9 and 62847 IMDB votes. It is a comedy and romance movie. Source: movie_titles table.
> Finished chain.


Again, It seems to have performed well.

#### Using Agents for querying database tables inplace of chains.

In [15]:
import os
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import AgentExecutor


toolkit = SQLDatabaseToolkit(db=db)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

agent_executor.run("List the tables in the database")



> Entering new AgentExecutor chain...


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI.


Action: list_tables_sql_db
Action Input: ""
Observation: movie_conversations, movie_titles, movie_characters, movie_script_urls, movie_lines
Thought: I now know the tables in the database
Final Answer: The tables in the database are movie_conversations, movie_titles, movie_characters, movie_script_urls, and movie_lines.

> Finished chain.


'The tables in the database are movie_conversations, movie_titles, movie_characters, movie_script_urls, and movie_lines.'

In [11]:
# Now running the same query using sql agent which earlier failed for dbchain without modifying the prompt.
agent_executor.run("which movie has most conversations ? ")



> Entering new AgentExecutor chain...
Action: list_tables_sql_db
Action Input: ""
Observation: movie_conversations, movie_titles, movie_characters, movie_script_urls, movie_lines
Thought: I should look at the schema of the movie_conversations table to see what columns I can query.
Action: schema_sql_db
Action Input: "movie_conversations"
Observation: 
CREATE TABLE movie_conversations (
	character_id_of_first_character TEXT, 
	character_id_of_second_character TEXT, 
	movie_id TEXT, 
	movie_line_id TEXT, 
	FOREIGN KEY(movie_id) REFERENCES movie_titles (movie_id), 
	FOREIGN KEY(character_id_of_first_character) REFERENCES movie_characters (character_id), 
	FOREIGN KEY(character_id_of_second_character) REFERENCES movie_characters (character_id), 
	FOREIGN KEY(movie_line_id) REFERENCES movie_lines (movie_line_id)
)

/*
2 rows from movie_conversations table:
character_id_of_first_character	character_id_of_second_character	movie_id	movie_line_id
u0	u2	m0	L194
u0	u2	m0	L195
*/
Thought: I shou

'The movie with the most conversations is m289 with 1530 conversations.'

In [14]:
# It worked but gave us slightly different answer like movie ids instead of title..
agent_executor.run("what are the names of top 4 movies which have most conversations along with their counts ? ")



> Entering new AgentExecutor chain...


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI.


Action: list_tables_sql_db
Action Input: ""
Observation: movie_conversations, movie_titles, movie_characters, movie_script_urls, movie_lines
Thought: I should check the schema of the movie_conversations table to see what columns I can query.
Action: schema_sql_db
Action Input: "movie_conversations"
Observation: 
CREATE TABLE movie_conversations (
	character_id_of_first_character TEXT, 
	character_id_of_second_character TEXT, 
	movie_id TEXT, 
	movie_line_id TEXT, 
	FOREIGN KEY(movie_id) REFERENCES movie_titles (movie_id), 
	FOREIGN KEY(character_id_of_first_character) REFERENCES movie_characters (character_id), 
	FOREIGN KEY(character_id_of_second_character) REFERENCES movie_characters (character_id), 
	FOREIGN KEY(movie_line_id) REFERENCES movie_lines (movie_line_id)
)

/*
2 rows from movie_conversations table:
character_id_of_first_character	character_id_of_second_character	movie_id	movie_line_id
u0	u2	m0	L194
u0	u2	m0	L195
*/
Thought: I can query the movie_conversations table to get

"The top 4 movies with the most conversations are 'Jackie Brown' (1214 conversations), 'Casino' (1530 conversations), 'Chill Factor' (1398 conversations), and 'Clerks.' (1286 conversations)."